In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/social-media-ad-dataset/social_media_ad_optimization.csv


# Predict ad engagement & conversions and
# Optimize delivery strategies for higher ROI.

In [2]:
import pandas as pd
train=pd.read_csv('/kaggle/input/social-media-ad-dataset/social_media_ad_optimization.csv')

In [3]:
train.head(5)

,user_id,age,gender,location,interests,ad_id,ad_category,ad_platform,ad_type,impressions,clicks,conversion,time_spent_on_ad,day_of_week,device_type,engagement_score
0,U0001,58,M,USA,Food,A0001,Sportswear,Facebook,Image,3,0,0,3.38,Friday,Mobile,0.02
1,U0002,55,F,USA,Tech,A0002,Electronics,Facebook,Image,9,9,1,6.77,Saturday,Tablet,0.93
2,U0003,52,F,UK,Gaming,A0003,Luggage,Instagram,Image,13,12,1,13.26,Wednesday,Mobile,0.93
3,U0004,31,F,USA,Tech,A0004,Gadgets,Facebook,Video,14,5,0,24.41,Saturday,Desktop,0.28
4,U0005,52,M,India,Tech,A0005,Luggage,Instagram,Carousel,10,5,0,21.43,Monday,Tablet,0.35


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           500 non-null    object 
 1   age               500 non-null    int64  
 2   gender            500 non-null    object 
 3   location          500 non-null    object 
 4   interests         500 non-null    object 
 5   ad_id             500 non-null    object 
 6   ad_category       500 non-null    object 
 7   ad_platform       500 non-null    object 
 8   ad_type           500 non-null    object 
 9   impressions       500 non-null    int64  
 10  clicks            500 non-null    int64  
 11  conversion        500 non-null    int64  
 12  time_spent_on_ad  500 non-null    float64
 13  day_of_week       500 non-null    object 
 14  device_type       500 non-null    object 
 15  engagement_score  500 non-null    float64
dtypes: float64(2), int64(4), object(10)
memory u

In [5]:
train.describe()

,age,impressions,clicks,conversion,time_spent_on_ad,engagement_score
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,39.410000,8.282000,4.520000,0.470000,14.072300,0.520220
std,12.400219,4.217351,3.727383,0.499599,9.088768,0.306296
min,18.000000,1.000000,0.000000,0.000000,0.460000,0.000000
25%,28.000000,4.750000,1.000000,0.000000,4.990000,0.270000
50%,39.000000,9.000000,4.000000,0.000000,13.285000,0.550000
75%,51.000000,12.000000,7.000000,1.000000,21.732500,0.772500
max,60.000000,15.000000,15.000000,1.000000,29.950000,1.000000


In [6]:
train.shape

(500, 16)

In [7]:
train['gender'].value_counts(normalize=True)

gender
M        0.356
Other    0.338
F        0.306
Name: proportion, dtype: float64

In [8]:
train['ad_platform'].value_counts(normalize=True)

ad_platform
Instagram    0.508
Facebook     0.492
Name: proportion, dtype: float64

In [9]:
train['ad_type'].value_counts(normalize=True)

ad_type
Image       0.356
Video       0.336
Carousel    0.308
Name: proportion, dtype: float64

In [10]:
train['location'].value_counts(normalize=True)

location
Australia    0.202
Germany      0.196
India        0.184
Canada       0.150
UK           0.144
USA          0.124
Name: proportion, dtype: float64

In [11]:
train['day_of_week'].value_counts(normalize=True)

day_of_week
Wednesday    0.172
Saturday     0.164
Sunday       0.152
Tuesday      0.146
Friday       0.124
Thursday     0.122
Monday       0.120
Name: proportion, dtype: float64

In [12]:
train['device_type'].value_counts(normalize=True)

device_type
Desktop    0.342
Mobile     0.338
Tablet     0.320
Name: proportion, dtype: float64

In [13]:
train['ad_category'].value_counts(normalize=True)

ad_category
Gadgets            0.182
Electronics        0.176
Luggage            0.176
Food & Beverage    0.170
Apparel            0.152
Sportswear         0.144
Name: proportion, dtype: float64

In [14]:
train['interests'].value_counts(normalize=True)

interests
Tech       0.186
Travel     0.180
Fitness    0.168
Food       0.166
Gaming     0.156
Fashion    0.144
Name: proportion, dtype: float64

In [15]:
train['location'].value_counts(normalize=True)

location
Australia    0.202
Germany      0.196
India        0.184
Canada       0.150
UK           0.144
USA          0.124
Name: proportion, dtype: float64

In [16]:
train['ctr']=train['clicks']/train['impressions']

In [17]:
train['cvr']=train['conversion']/train['clicks']

# CTR by device category

In [18]:
# Group by device_type and aggregate clicks + impressions
ctr_devicetype = (
    train.groupby('device_type')
         .agg({'clicks': 'sum', 'impressions': 'sum'})
         .reset_index()
)

# Compute CTR
ctr_devicetype['CTR'] = (ctr_devicetype['clicks'] / ctr_devicetype['impressions']) * 100

print(ctr_devicetype)


  device_type  clicks  impressions        CTR
0     Desktop     735         1413  52.016985
1      Mobile     758         1434  52.859135
2      Tablet     767         1294  59.273570


# CTR by device category

In [19]:
cvr_by_category = (
    train.groupby('ad_category')
         .agg({'clicks': 'sum', 'conversion': 'sum'})
         .reset_index()
)

# Compute CVR
cvr_by_category['CVR (%)'] = (cvr_by_category['conversion'] / cvr_by_category['clicks']) * 100

print(cvr_by_category)

       ad_category  clicks  conversion    CVR (%)
0          Apparel     322          33  10.248447
1      Electronics     420          41   9.761905
2  Food & Beverage     314          44  14.012739
3          Gadgets     404          39   9.653465
4          Luggage     407          38   9.336609
5       Sportswear     393          40  10.178117


# Engagement by day of week

In [20]:
# Group by day_of_week and compute mean engagement_score
engagement_by_day = (
    train.groupby('day_of_week')['engagement_score']
         .mean()
         .reset_index()
         .sort_values('engagement_score', ascending=False)
)

print(engagement_by_day)


  day_of_week  engagement_score
6   Wednesday          0.566860
3      Sunday          0.538421
5     Tuesday          0.531507
4    Thursday          0.505902
1      Monday          0.505167
2    Saturday          0.504390
0      Friday          0.469516


#Step 2: Feature Engineering

User Features: average CTR per user, preferred ad category.

Ad Features: CTR by category/platform/type.

Time Features: weekday vs weekend, time_spent_on_ad bucketed.

Interaction Features: age × category (which age groups prefer which ads).

In [21]:
train.head(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,user_id,age,gender,location,interests,ad_id,ad_category,ad_platform,ad_type,impressions,clicks,conversion,time_spent_on_ad,day_of_week,device_type,engagement_score,ctr,cvr
0,U0001,58,M,USA,Food,A0001,Sportswear,Facebook,Image,3,0,0,3.38,Friday,Mobile,0.02,0.000000,NaN
1,U0002,55,F,USA,Tech,A0002,Electronics,Facebook,Image,9,9,1,6.77,Saturday,Tablet,0.93,1.000000,0.111111
2,U0003,52,F,UK,Gaming,A0003,Luggage,Instagram,Image,13,12,1,13.26,Wednesday,Mobile,0.93,0.923077,0.083333
3,U0004,31,F,USA,Tech,A0004,Gadgets,Facebook,Video,14,5,0,24.41,Saturday,Desktop,0.28,0.357143,0.000000
4,U0005,52,M,India,Tech,A0005,Luggage,Instagram,Carousel,10,5,0,21.43,Monday,Tablet,0.35,0.500000,0.000000


# CTR -prefered ad category

In [22]:
ctr_adcategory = (
    train.groupby('ad_category')
         .agg({'clicks': 'sum', 'impressions': 'sum'})
         .reset_index()
)

# Compute CTR
ctr_adcategory['CTR'] = (ctr_devicetype['clicks'] / ctr_devicetype['impressions']) * 100

print(ctr_devicetype)

  device_type  clicks  impressions        CTR
0     Desktop     735         1413  52.016985
1      Mobile     758         1434  52.859135
2      Tablet     767         1294  59.273570


In [23]:
train['day_type'] = train['day_of_week'].apply(
    lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')